In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [2]:
def dadosPagina(sopa, zona):

    lista_precos = []
    lista_areas = []
    lista_quartos = []

    sopa_precos = sopa.find_all('span', class_='price__fraction')
    if sopa_precos == []:
        sopa_precos = sopa.find_all('span', class_='price-tag-fraction')
        
    for preco in sopa_precos:
        lista_precos.append(preco.text.replace('.', ''))
        
    sopa_string = sopa.find_all('div', class_='item__attrs')
    if sopa_string != []:
        for elemento in sopa_string:
            if "|" in str(elemento):
                area = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                quarto = re.search(r'\d+', elemento.text.split(' | ')[1]).group()
            elif "quarto" in str(elemento):
                quarto = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                area = np.nan
            else:
                area = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                quarto = np.nan
            
            lista_areas.append(area)
            lista_quartos.append(quarto)        
    else:
        sopa_string = sopa.find_all('li', class_='ui-search-card-attributes__attribute')
        lista_areas_quartos = []
        i = 0
        if 'quarto' not in str(sopa_string[-1]):
            sopa_string.append('nada')
            
        while i <= len(sopa_string) -1 :
            if 'm²' not in str(sopa_string[i]):
                sopa_string.insert(i, "nada")
                lista_areas_quartos.append(str(sopa_string[i]) + ' , ' + re.search(r'\d+', str(sopa_string[i+1])).group())
                i += 2
            elif 'quarto' not in str(sopa_string[i+1]) and str(sopa_string[i+1]) != 'nada':
                sopa_string.insert(i+1, "nada")
                lista_areas_quartos.append(re.search(r'\d+', str(sopa_string[i])).group() + ' , ' + str(sopa_string[i+1]))
                i += 2
            else:
                if str(sopa_string[i+1]) != 'nada':
                    lista_areas_quartos.append(re.search(r'\d+', str(sopa_string[i])).group() + ' , ' + re.search(r'\d+', str(sopa_string[i+1])).group())
                    i += 2
                else:
                    lista_areas_quartos.append(re.search(r'\d+', str(sopa_string[i])).group() + ' , nada')
                    i += 2
                    
        for elemento in lista_areas_quartos:
            lista_areas.append(elemento.split(',')[0])
            lista_quartos.append(elemento.split(',')[1])   
    

    lista_zona = [zona]*len(lista_precos)
    
    dados = {'zona': lista_zona, 'preco': lista_precos, 'area': lista_areas, 'quartos': lista_quartos}
    df = pd.DataFrame(dados)
    
    return df
    


In [3]:
def zonaPaginas(zona):

    lista_df = []
    lista_urls = []

    url = 'https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-{}/'
    if requests.get(url.format(zona)).status_code == 200:
        conteudo = requests.get(url.format(zona)).content
        sopa = BeautifulSoup(conteudo)
        lista_df.append(dadosPagina(sopa, zona))
        sleep(3)
        links = sopa.find_all("li", class_= 'andes-pagination__button')
        pagina = 1 

        for link in links:
            re_link = 'href="(.*)">{}</a>'.format(pagina)
            lista_urls.append(re.findall(re_link, str(link)))
            pagina +=1
        
        for i in range(1, len(lista_urls) -1):
            if lista_urls is not []:
                link_url = lista_urls[i][0]
                print(link_url)
                conteudo_link = requests.get(link_url).content
                sopa = BeautifulSoup(conteudo_link)
                lista_df.append(dadosPagina(sopa, zona))
                sleep(3)
    else:
        print('Página fora do ar')

    return lista_df


In [4]:
localidades = ['norte', 'sul', 'oeste']

lista_dfs = []
for local in localidades:
    print(local)
    df = zonaPaginas(local)
    lista_dfs += df



norte
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-norte/_Desde_49
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-norte/_Desde_97
sul
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-sul/_Desde_97
oeste
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-oeste/_Desde_49
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-oeste/_Desde_97
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-oeste/_Desde_145
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-oeste/_Desde_193
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-oeste/_Desde_241
https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-jane

In [5]:
lista_dfs

      2300    95        3
 19  norte         800    40        1
 20  norte        3000  nada        2
 21  norte        7000   240     nada
 22  norte        2300    81        2
 23  norte        2300    90        2
 24  norte        3000   120        3
 25  norte        8000   350        6
 26  norte        3000   188        4
 27  norte        2700    70        2
 28  norte        7000   250        7
 29  norte       18000     1        4
 30  norte          89    70        2
 31  norte        6000   200        2
 32  norte        1900    60        3
 33  norte        1900    76        2
 34  norte        1200    52        2
 35  norte        3000   127        2
 36  norte        1700    70        2
 37  norte        4000   233        2
 38  norte        3500   220     nada
 39  norte        1800    80        2
 40  norte        3000   239        2
 41  norte        2200    76        2
 42  norte         800    60        1
 43  norte        6000   198        2
 44  norte         800  

In [6]:
dados_raw = pd.concat(lista_dfs)

In [7]:
dados_raw

,zona,preco,area,quartos
0,norte,1048049932,220,4
1,norte,1000,65,2
2,norte,900,44,2
3,norte,750,40,1
4,norte,3000,300,7
...,...,...,...,...
36,oeste,1250,60,2
37,oeste,400,60,2
38,oeste,650,50,1
39,oeste,600,36,1


In [8]:
dados_raw.to_csv('dadosMl.csv', index=False)